## Solvency demo

Here we create a demo of a solvency calculation in the manner of [summa-solvency](https://github.com/summa-dev/summa-solvency). The aim here is to demonstrate the use of the new polycommit method detailed [here](https://blog.ezkl.xyz/post/commits/).

In this setup:
- the commitments to users, respective balances, and total balance are known are publicly known to the prover and verifier.
- We leave the outputs of the model as public as well (known to the  verifier and prover).

The circuit calculates the total sum of the balances, and checks that it is less than the total balance which is precommited to.

In [1]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pytest"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

# uncomment to enable logging
# import logging
# FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
# logging.basicConfig(format=FORMAT)
# logging.getLogger().setLevel(logging.DEBUG)


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch

class Circuit(nn.Module):
    def __init__(self):
        super(Circuit, self).__init__()

    def forward(self, users, balances, total):
        nil = torch.nn.Parameter(torch.tensor([0.0]))
        # calculate the total balance across all users second term will be ignored by the optimizer but will force it to be included in a separate col for commitment
        balances = torch.sum(balances, dim=1) + nil * users
        # now check if the total balance is less than the total
        return (balances[:,0] <= total)


circuit = Circuit()

# Train the model as you like here (skipped for brevity)



In [2]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

We create dummy data here for the sake of demonstration. In a real world scenario, the data would be provided by the users, and the commitments would be made by some trusted party.

The users are generated as hashes of the integers 0 to 9. The balances are generated as integers between 0 and 10.

The total balance is the sum of the balances.

In [3]:


user_preimages = [0.0, 1.0, 2.0, 3.0, 4.0, 9.0]
balances = torch.tensor([0, 2, 3, 4, 5, 10])
balances = balances.reshape(1, 6)


# Create an empty list to store the hashes of float -- which I guess we'll call the users here
users = []

# Loop through each element in the y tensor
for e in user_preimages:
    # Apply the custom function and append the result to the list
    users.append(ezkl.poseidon_hash([ezkl.float_to_felt(e, 0)])[0])

users_t = torch.tensor(user_preimages)
users_t = users_t.reshape(1, 6)

total = torch.tensor([25])
total = total.reshape(1, 1)

# Flips thegraph into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      (users_t,balances,total),                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=17,          # the ONNX version to export the model to
                      do_constant_folding=False,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})



data_array_x = users
data_array_y = ((balances).detach().numpy()).reshape([-1]).tolist()
data_array_z = ((total).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array_x, data_array_y, data_array_z])


    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


/state/partition1/job-60084051/ipykernel_2668910/3345491920.py:36: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  nil = torch.nn.Parameter(torch.tensor([0.0]))


This is where the magic happens. We define our `PyRunArgs` objects which contains the visibility parameters for out model.
- `input_visibility` defines the visibility of the model inputs
- `param_visibility` defines the visibility of the model weights and constants and parameters
- `output_visibility` defines the visibility of the model outputs

There are currently 5 visibility settings:
- `public`: known to both the verifier and prover (a subtle nuance is that this may not be the case for model parameters but until we have more rigorous theoretical results we don't want to make strong claims as to this).
- `private`: known only to the prover
- `hashed`: the hash pre-image is known to the prover, the prover and verifier know the hash. The prover proves that the they know the pre-image to the hash.
- `encrypted`: the non-encrypted element and the secret key used for decryption are known to the prover. The prover and the verifier know the encrypted element, the public key used to encrypt, and the hash of the decryption hey. The prover proves that they know the pre-image of the hashed decryption key and that this key can in fact decrypt the encrypted message.
- `polycommit`: unblinded advice column which generates a kzg commitment. This doesn't appear in the instances of the circuit and must instead be modified directly within the proof bytes.  

Here we create the following setup:
- `input_visibility`: "polycommit"
- `param_visibility`: "public"
- `output_visibility`: public

We encourage you to play around with other setups :)

Shoutouts:

- [summa-solvency](https://github.com/summa-dev/summa-solvency) for their help with the poseidon hashing chip.
- [timeofey](https://github.com/timoftime) for providing inspiration in our developement of the el-gamal encryption circuit in Halo2.

In [4]:
run_args = ezkl.PyRunArgs()
# "polycommit" means that the output of the hashing is not visible to the verifier and is instead fed into the computational graph
run_args.input_visibility = "polycommit"
run_args.ignore_range_check_inputs_outputs = True
# the parameters are public
run_args.param_visibility = "fixed"
# the output is public (this is the inequality test)
run_args.output_visibility = "public"
run_args.variables = [("batch_size", 1)]
# never rebase the scale
run_args.scale_rebase_multiplier = 1000
# logrows
run_args.logrows = 11
run_args.lookup_range = (-1000,1000)
run_args.input_scale = 0
run_args.param_scale = 0


# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True


In [5]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

low scale values (<8) may impact precision


In [6]:
# srs path
res = await ezkl.get_srs( settings_path)

We'll generate two proofs, one with the correct total balance, and one with an incorrect total balance.

## Correct total balance

The data file above has a total balance of above the user total balance. We'll generate a proof with this total balance.

In [7]:
# setup keypair
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,

    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

Using 9 columns for range-check.
Using 9 columns for range-check.


In [8]:
!export RUST_BACKTRACE=1

witness_path = "witness.json"

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path, vk_path)
assert os.path.isfile(witness_path)

# we force the output to be 1 this corresponds to the solvency test being true -- and we set this to a fixed vis output
# this means that the output is fixed and the verifier can see it but that if the input is not in the set the output will not be 0 and the verifier will reject
witness = json.load(open(witness_path, "r"))
witness["outputs"][0] = [ezkl.float_to_felt(1.0, 0)]
json.dump(witness, open(witness_path, "w"))

Using 9 columns for range-check.


In [9]:
proof_path = os.path.join('proof.json')
# proof path
res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,

        "single",
    )

assert os.path.isfile(proof_path)

print(res)


Using 9 columns for range-check.
Using 9 columns for range-check.


{'instances': [['0100000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x0b68f7751f17d3d6d810ba8cc091ffae20703dd20972a686fd2676cf914315262d4f926403d1f2a8e8a09f012e18cced6f9f956b1f4a39b842b81548ca4ee17408dacea1244bcf5cfb726b9e48d182f6dc25686951feb6b5607a690ffdd1d2fb1ae7f814fd1529d1cd67073aa2f6350feed333496d9f75197c4073f4f8a8dc4527cc6e3c08a6b346796fd27dd64d35e647e5f6c0a3498267a78fe3b97b0a4b1201e2ad516f0c83bbe86f91b9480b171c10ef02df2e184b88c6a886dde6058e742168b92a6d5f323e53c55fd073961a8e7d50caa553884d8aeccb530e2890d0091282eaa50f953c2c237f24aba1300ca6d56847f74a1e1c41b6a600fb0b93bcde122b5e29c9a0b8295b6457cc5cf3d25fcfa7a4c55e0132ccfe2b4770bc9ab34c0464bbe8e8115c0c378578543855c7e16e5d2d07e17f4132db2aca0b8ef77f87236e6f31f2f6e86ea2e6c4e3952bbc162c6c9b96361789781786a4e513540d5e228bd8d213bacaff7ffb97787e58a959d7192cd20957672a4e13a88ad987035708c81b806ba6cbdc4b10b0257f997b87b66433ff64142d38446056db50e34ee5255ec3dc0e67bbc775b75ebff51d2c9eb74afe12135fc995a509f73507e56d251d36a5

- now we swap the commitments of the proof as a way to demonstrate that the proof is valid given some public inputs
- this is just for testing purposes and would require fetching public commits from the blockchain or some other source
- see https://blog.ezkl.xyz/post/commits/ for more details

In [10]:


res = ezkl.swap_proof_commitments(proof_path, witness_path)


In [11]:
# verify the proof
res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,

    )
assert res == True

Using 9 columns for range-check.


### Faulty proof

We'll generate a proof with a total balance of 10. This is below the user total balance.

In [12]:
# now generate a truthy input + witness file (x input not in the set)
import random

data_path_truthy = os.path.join('input.json')
data = json.load(open(data_path, 'r' ))
data['input_data'][2] = [10]

data_path_faulty = os.path.join('input_faulty.json')
# Serialize data into file:
json.dump( data, open(data_path_faulty, 'w' ))


In [13]:
# now generate the witness file

res = await ezkl.gen_witness(data_path_faulty, compiled_model_path, witness_path, vk_path)
assert os.path.isfile(witness_path)

# we force the output to be 1 this corresponds to the solvency test being true -- and we set this to a fixed vis output
# this means that the output is fixed and the verifier can see it but that if the input is not in the set the output will not be 0 and the verifier will reject
witness = json.load(open(witness_path, "r"))
witness["outputs"][0] = [ezkl.float_to_felt(1.0, 0)]
json.dump(witness, open(witness_path, "w"))


Using 9 columns for range-check.


In [14]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,

        "single",
    )

print(res)
assert os.path.isfile(proof_path)

Using 9 columns for range-check.
Using 9 columns for range-check.


{'instances': [['0100000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x0b68f7751f17d3d6d810ba8cc091ffae20703dd20972a686fd2676cf914315262d4f926403d1f2a8e8a09f012e18cced6f9f956b1f4a39b842b81548ca4ee17408dacea1244bcf5cfb726b9e48d182f6dc25686951feb6b5607a690ffdd1d2fb1ae7f814fd1529d1cd67073aa2f6350feed333496d9f75197c4073f4f8a8dc4504714919575dfb57f4c04bdd4d68b4d2ef8dc76ba7f13eb0853a2e4b8b51f7c41fd30722bafb3ee77ad33db85c7e7a68ae902524cf88c760dbefd26ee527df7e230fb76df168205966b3401de3f8aef94a6512794fd7b0b2741cb2f6c63e6f512b789722a0acca351d827757514dd05b0dfdc241124f52c4cd001c69c558295629f5171610dedaa8e5f3dc55cb82c7e4082011225dafe38ab9fe3bf1927d6f252cd51ab05522f6e73cdffc8661c21ef98b7ba4b0954caded1d7868cf6c1d14551d88fbc8000c9fc5bdbe86b39baddf7d6e30d66ed8217d74d1088c3c7b3a99311ece2ccaba63840db5602e71760f7464c07e4fe22170eeac91e2681e1a6e46522cef32f957aa1cd92ce1c140ad7391a06cc86b26da5f633d8c8b41d291ce5c572a3674275f8c0af2350f845de7403838ba500501891cd4b4ed29443747d13050222153

In [15]:
res = ezkl.swap_proof_commitments(proof_path, witness_path)


Now we test that verification fails

In [17]:
import pytest

def test_verification():
    with pytest.raises(RuntimeError, match='Failed to run verify: \\[halo2\\] The constraint system is not satisfied'):
        ezkl.verify(
            proof_path,
            settings_path,
            vk_path,
        )

# Run the test function
test_verification()

Using 9 columns for range-check.
